In [1]:
# these are the commands that everyone needs

In [3]:
from pymongo import MongoClient
import math
from geopy.distance import vincenty
from tqdm import tqdm
import json
client = MongoClient()
db = client.yelp

In [ ]:
user_location = {}
category_ave_distance_by_city = {}

def get_user_living_location(businesses_in_specified_city):
    all_reviews_by_users = db.reviews.find({"business_id": {"$in": businesses_in_specified_city}})
    users_in_city = all_reviews_by_users.distinct("user_id")
    
    for user in tqdm(users_in_city):
        businesses_in_city_reviewed_by_user = db.reviews.find({"business_id": {"$in": businesses_in_specified_city}, "user_id": user}).distinct("business_id")
        latitude = 0.0
        longitude = 0.0
        count = len(businesses_in_city_reviewed_by_user)
        business_obj_in_city_reviewed_by_user = db.businesses.find({"business_id": {"$in": businesses_in_city_reviewed_by_user}})
        for b in business_obj_in_city_reviewed_by_user:
            latitude += float(b['latitude'])
            longitude += float(b['longitude'])
        if count > 8:
            user_location[user] = [latitude/count, longitude/count]
        else:
            continue

In [ ]:
# import all cities to the database
# run exactly once (takes ≈ 24 hours)
cities = db.businesses.find().distinct("city")

for city in cities:
    businesses = db.businesses.find({"city": city}).distinct("business_id")
    user_location = {}
    get_user_living_location(businesses)
    print city, len(user_location)

    for user in tqdm(user_location):
        entry = {'city': city, 'user_id': user, "living_location": user_location[user]}
        db.userAddress.insert_one(entry)

In [ ]:
def get_users_by_reviews(bCategory, newKey, city, users_in_city):
    for user in users_in_city:
        category_reviews_by_user = db.reviews.find({"user_id": user, "business_id": {"$in": bCategory.distinct("business_id")}})
        short_b = category_reviews_by_user.distinct("business_id")

        item = db.userAddress.find_one({'city': city, 'user_id': user})
        home = item['living_location'] 

        short_b2 = db.businesses.find({"business_id": {"$in": short_b}})
        for b in short_b2:
            business_loc = [float(b['latitude']), float(b['longitude'])]
            miles_dist = vincenty(home, business_loc).miles
        
            temp = category_ave_distance_by_city[newKey]
            category_ave_distance_by_city[newKey] = (temp[0] + miles_dist, temp[1] + 1)

In [4]:
# also run exactly once (takes forever also)
category_ave_distance_by_city = {}
cities = db.userAddress.find().distinct('city')
for city in tqdm(cities):
    users_in_city = db.userAddress.find({'city': city}).distinct('user_id')
    industries = db.businesses.find({"city": city}).distinct("categories")
    businesses = db.businesses.find({"city": city}).distinct("business_id")
    for industry in industries:
        category = ["{}".format(industry)]
        businesses_by_category = db.businesses.find({"business_id": {"$in": businesses}, "categories": {"$elemMatch": {"$in": category}}})
        size = len(businesses_by_category.distinct("business_id"))
        if size > 8:
            newKey = city + industry
            category_ave_distance_by_city[newKey] = (0.0, 0.0)
            get_users_by_reviews(businesses_by_category, newKey, city, users_in_city)
        else:
            continue

        temp = category_ave_distance_by_city[newKey]
        if temp[1] != 0:
            category_ave_distance_by_city[newKey] = (temp[0]/temp[1], 1)
            entry = {'city': city, 'industry': industry, 'ave_travel_dist': category_ave_distance_by_city[newKey][0]}
            print entry
            db.cityIndustryDist.insert_one(entry)
            
#     print "{:25}{}".format("items in category: ",category_ave_distance_by_city[newKey][1])
#     for c in category_ave_distance_by_city:
#         temp = category_ave_distance_by_city[c]
#         if temp[1] != 0:
#             category_ave_distance_by_city[c] = (temp[0]/temp[1], 1)

#     print "{:25}{}".format(newKey, category_ave_distance_by_city[newKey])

  0%|          | 0/185 [00:00<?, ?it/s]


NameError: name 'get_users_by_reviews' is not defined